In [1]:
import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./2.subategory.xlsx',
                       header=0,
                       dtype=str).fillna('')

length = len(input_)

print('总数量：' + str(length))
input_['Subcategory'].tolist()

总数量：330


['RockAuto Apparel',
 'RockAuto Gift',
 'ATV / UTV Belt',
 'Automotive V-Belt',
 'Fluid / Chemical',
 'Heavy Duty / Industrial V-Belt',
 'Light Duty FHP / Lawn & Garden V-Belt',
 'Pulley & Tensioner',
 'Refrigeration V-Belt',
 'Snowmobile Belt',
 'Tool',
 'Accessory Lighting',
 'Antenna',
 'Back Up / Reverse Lamp',
 'Body Panel',
 'Bumper / Pad',
 'Cargo Management',
 'Clearance / Side Marker Lamp',
 'Door',
 'Fender',
 'Fluid / Chemical',
 'Fog / Driving Lamp',
 'Grille',
 'Hardware',
 'Headlamp',
 'Hood',
 'Lamp Bezel',
 'License Plate',
 'Lift Support',
 'Mud Flap',
 'Outside Mirror',
 'Overspray Protection',
 'Painting',
 'Parking / Turn Signal Lamp',
 'Polishing',
 'Recovery Gear',
 'Reflector',
 'Retainer / Clip',
 'Running Board',
 'Stop, Turn, & Tail Lamp',
 'Tape',
 'Tool',
 'Towing - Fifth Wheel',
 'Towing - Flat / Vehicle',
 'Towing - Trailer',
 'Towing Enhancement',
 'Trim / Molding',
 'Weatherstrip',
 'Air Brake',
 'Axle/Spindle',
 'Brake Line',
 'Brake Shoe',
 'Caliper',


In [3]:
crawler_status = 'error'

print(crawler_status)

error


In [4]:
from urllib.parse import quote
import requests
import json

In [5]:
i = 0

jsn = '{"jsn":{"tab":"extras","groupname":"' + input_.loc[i, 'Category'] + '","subgroupname":"' + input_.loc[i, 'Subcategory'] + '","nodetype":"subgroupname"}}'

payload = {'func': 'navnode_fetch',
           'payload': jsn,
           'api_json_request': '1'}

resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                     data=payload).text

resp = json.loads(resp)

resp

{'html_fill_sections': {'navchildren[]': '<div class="ranavnode" id="nav[__GIP__1__]"><input type="hidden" autocomplete="off" id="jsn[__GIP__1__]" value="{&quot;groupindex&quot;:&quot;__GIP__1__&quot;,&quot;tab&quot;:&quot;extras&quot;,&quot;idepth&quot;:1,&quot;groupname&quot;:&quot;Apparel &amp; Gifts&quot;,&quot;subgroupname&quot;:&quot;RockAuto Apparel&quot;,&quot;parttype&quot;:&quot;131&quot;,&quot;nodetype&quot;:&quot;parttype&quot;}" /><input type="hidden" autocomplete="off" name="navnodeunique[extras-apparel &amp; gifts-rockauto apparel-131]" id="navnodeunique[extras-apparel &amp; gifts-rockauto apparel-131]" value="__GIP__1__" /><div id="nav_o[__GIP__1__]" class="ranavouter"><div class="inner"><table class="tbl"  cellpadding="0" cellspacing="0"><tr><td class="niconspace nvlineicon"></td><td class="niconspace ncollapsedicon" id="navicon[__GIP__1__]"><a href="/en/tools/apparel+&amp;+gifts,rockauto+apparel,hat,131" onclick="cataloglite.LinkIntercept_ToggleNavNode(&quot;__GIP__1_

In [6]:
from bs4 import BeautifulSoup

In [7]:
html = resp['html_fill_sections']['navchildren[]']

soup = BeautifulSoup(html, 'lxml')

with open('./type.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <div class="ranavnode" id="nav[__GIP__1__]">
   <input autocomplete="off" id="jsn[__GIP__1__]" type="hidden" value='{"groupindex":"__GIP__1__","tab":"extras","idepth":1,"groupname":"Apparel &amp; Gifts","subgroupname":"RockAuto Apparel","parttype":"131","nodetype":"parttype"}'/>
   <input autocomplete="off" id="navnodeunique[extras-apparel &amp; gifts-rockauto apparel-131]" name="navnodeunique[extras-apparel &amp; gifts-rockauto apparel-131]" type="hidden" value="__GIP__1__"/>
   <div class="ranavouter" id="nav_o[__GIP__1__]">
    <div class="inner">
     <table cellpadding="0" cellspacing="0" class="tbl">
      <tr>
       <td class="niconspace nvlineicon">
       </td>
       <td class="niconspace ncollapsedicon" id="navicon[__GIP__1__]">
        <a href="/en/tools/apparel+&amp;+gifts,rockauto+apparel,hat,131" onclick='cataloglite.LinkIntercept_ToggleNavNode("__GIP__1__"); return false;' tabindex="-1">
         <img alt="Toggle" src="/Images/1pxtransparent.png"/>
   

In [8]:
from lxml import etree

In [9]:
html = etree.HTML(str(soup))

list_type = [type_.strip() for type_ in html.xpath('//td[@class="nlabel"]/a/text()')]

list_type

['Hat', 'Hoodie', 'T-Shirt']

In [10]:
list_url = ['https://www.rockauto.com'+url.strip() for url in html.xpath('//td[@class="niconspace ncollapsedicon"]/a/@href')]

list_url

['https://www.rockauto.com/en/tools/apparel+&+gifts,rockauto+apparel,hat,131',
 'https://www.rockauto.com/en/tools/apparel+&+gifts,rockauto+apparel,hoodie,1002098',
 'https://www.rockauto.com/en/tools/apparel+&+gifts,rockauto+apparel,t-shirt,256']

In [11]:
crawler_status = 'ok'

print(crawler_status)

ok


In [12]:
if crawler_status == 'ok':
    df_temp = pd.DataFrame({'Tab_Label': input_.loc[i, 'Tab_Label'],
                            'No.': int(input_.loc[i, 'No.']),
                            'Category': input_.loc[i, 'Category'],
                            'Subcategory': input_.loc[i, 'Subcategory'],
                            'Type': list_type,
                            'Type_Code': [url.split(',')[-1] for url in list_url],
                            'Url': list_url})
    
    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
else:
    df_temp = pd.DataFrame([{'Tab_Label': input_.loc[i, 'Tab_Label'],
                             'No.': int(input_.loc[i, 'No.']),
                             'Category': input_.loc[i, 'Category'],
                             'Subcategory': input_.loc[i, 'Subcategory'],
                             'Url': input_.loc[i, 'Url']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
    
print(len(output_correct), len(output_error))

3 0
